In [1]:
import pandas as pd
import json


### Arranging the Players DataFrame

In [2]:
df_player = pd.read_csv('/Users/pierrecanadas/code/pirroux/personal_projects/nbadata/01-Project-Setup/data/data_all_players.csv')

### Extracting players info from dictionnary into separate columns

In [3]:
data = df_player
df_player = pd.DataFrame(data)

# Apply lambda function to create a new 'id_column'
df_player['player_team_id'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('id'))
df_player['player_team_abb'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('abbreviation'))
df_player['player_team_city'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('city'))
df_player['player_team_conf'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('conference'))
df_player['player_team_div'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('division'))
df_player['player_team_full_name'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('full_name'))
df_player['player_team_name'] = df_player['team'].apply(lambda x: json.loads(x.replace("'", "\"")).get('name'))

In [4]:
df_player.drop(columns=["team", "weight_pounds", "height_feet", "height_inches"], inplace=True)

### Arranging the Game DataFrame

In [5]:
df_games = pd.read_csv('/Users/pierrecanadas/code/pirroux/personal_projects/nbadata/01-Project-Setup/data/games_data.csv')

In [6]:
### Extracting game info from dictionnary into separate columns

In [7]:
data_games = df_games
df_games = pd.DataFrame(data_games)

# Apply lambda function to create a new 'id_column'
df_games['new_home_team_id'] = df_games['id'].apply(lambda x: json.loads(x.replace("'", "\"")).get('id'))
df_games['new_vis_team_id'] = df_games['time'].apply(lambda x: json.loads(x.replace("'", "\"")).get('id'))


In [8]:
df_games.drop(columns=["id", "time"], inplace=True)

In [9]:
df_games.rename(columns={"date": "score_home",
                        "postseason": "score_vis"}, inplace=True)

In [10]:
# Managing a MultiIndex in the DataFrame
df_games = df_games.reset_index()
df_games.rename(columns={"level_0":"game_id",
                        "level_1": "game_date"}, inplace=True)
df_games["game_date"] = pd.to_datetime(df_games["game_date"])

### Arranging the Stats DataFrame

In [11]:
import ast

In [12]:
df_stats = pd.read_csv('/Users/pierrecanadas/code/pirroux/personal_projects/nbadata/01-Project-Setup/data/all_stats.csv')

In [13]:
# Transform columns with lists (dictionnaries looking like lists) into dictionaries
df_stats['game'] = df_stats['game'].apply(ast.literal_eval)
df_stats['team'] = df_stats['team'].apply(ast.literal_eval)
df_stats['player'] = df_stats['player'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)

In [14]:
# Extract game_id, player_id and team_id from the columns
df_stats['game_id']=df_stats['game'].apply(lambda x : x['id'] if 'id' in x else None)
df_stats['player_id'] = df_stats['player'].apply(lambda x: int(x.get('id')) if pd.notna(x) and 'id' in x and pd.notna(x.get('id')) else None).astype('Int64')
df_stats['team_id'] = df_stats['team'].apply(lambda x: int(x.get('id')) if pd.notna(x) and 'id' in x and pd.notna(x.get('id')) else None).astype('Int64')

### Extracting info from dictionnary into separate columns

In [15]:
df_stats['first_name'] = df_stats['player'].apply(lambda x: x.get('first_name') if pd.notna(x) and 'first_name' in x else None)
df_stats['last_name'] = df_stats['player'].apply(lambda x: x.get('last_name') if pd.notna(x) and 'last_name' in x else None)
df_stats['position'] = df_stats['player'].apply(lambda x: x.get('position') if pd.notna(x) and 'position' in x else None)
df_stats['abbreviation'] = df_stats['team'].apply(lambda x: x.get('abbreviation') if pd.notna(x) and 'abbreviation' in x else None)

### Remove ambiguous team id in player's data dictionnnary

In [16]:
def remove_key_from_dict(d, key):
    if isinstance(d, dict):
        d.pop(key, None)
    return d

key_to_remove = 'team_id'
df_stats['player'] = df_stats['player'].apply(lambda x: remove_key_from_dict(x, key_to_remove))

### Merging DataFrames together

In [17]:
# Merging the games and stats dataframes

In [144]:
merged_df = df_games.merge(df_stats, how='inner', on='game_id')

In [145]:
#merged_df.duplicated().sum()

In [146]:
df_player.rename(columns={"id": "player_id"}, inplace=True)
df_player.player_id = df_player.player_id.astype('Int64')

In [147]:
merged_df.rename(columns={"new_home_team_id":"home_team_id",
                  "new_vis_team_id": "vis_team_id",
                  })

,game_id,game_date,score_home,home_team_id,visitor_team_id,season,period,status,score_vis,home_team_id,vis_team_id,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,game,min,oreb,pf,player,pts,reb,stl,team,turnover,player_id,team_id,first_name,last_name,position,abbreviation
0,47179,2019-01-30,126,4,False,2018,Final,,94,2,4,1076654,4.0,0.0,0.0,0.250,4.0,1.0,0.545000,11.0,6.0,0.000000,0.0,0.0,"{'id': 47179, 'date': '2019-01-30', 'home_team...",26:38,2.0,3.0,"{'id': 33, 'first_name': 'Nicolas', 'height_fe...",13.0,2.0,2.0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",3.0,33,4,Nicolas,Batum,F-G,CHA
1,47179,2019-01-30,126,4,False,2018,Final,,94,2,4,1076655,0.0,0.0,2.0,1.000,2.0,2.0,1.000000,4.0,4.0,0.000000,0.0,0.0,"{'id': 47179, 'date': '2019-01-30', 'home_team...",27:23,0.0,0.0,"{'id': 482, 'first_name': 'Marvin', 'height_fe...",10.0,2.0,1.0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",0.0,482,4,Marvin,Williams,F,CHA
2,47179,2019-01-30,126,4,False,2018,Final,,94,2,4,1076656,2.0,1.0,4.0,0.000,0.0,0.0,0.333000,3.0,1.0,0.000000,0.0,0.0,"{'id': 47179, 'date': '2019-01-30', 'home_team...",13:34,1.0,1.0,"{'id': 48, 'first_name': 'Bismack', 'height_fe...",2.0,5.0,0.0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",1.0,48,4,Bismack,Biyombo,C,CHA
3,47179,2019-01-30,126,4,False,2018,Final,,94,2,4,1076657,1.0,0.0,5.0,0.400,5.0,2.0,0.286000,7.0,2.0,0.000000,0.0,0.0,"{'id': 47179, 'date': '2019-01-30', 'home_team...",23:10,0.0,2.0,"{'id': 267, 'first_name': 'Jeremy', 'height_fe...",6.0,5.0,0.0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",2.0,267,4,Jeremy,Lamb,G,CHA
4,47179,2019-01-30,126,4,False,2018,Final,,94,2,4,1076658,2.0,1.0,1.0,0.286,7.0,2.0,0.450000,20.0,9.0,1.000000,1.0,1.0,"{'id': 47179, 'date': '2019-01-30', 'home_team...",26:31,0.0,0.0,"{'id': 465, 'first_name': 'Kemba', 'height_fee...",21.0,1.0,1.0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",1.0,465,4,Kemba,Walker,G,CHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234533,1038294,2024-02-03,117,4,False,2023,Final,Final,129,7,17,14391931,0.0,0.0,0.0,0.000,0.0,0.0,1.000000,1.0,1.0,0.000000,0.0,0.0,"{'id': 1038294, 'date': '2024-02-03', 'home_te...",04,0.0,0.0,"{'id': 284, 'first_name': 'Robin', 'height_fee...",2.0,0.0,0.0,"{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil...",0.0,284,17,Robin,Lopez,C,MIL
234534,1038294,2024-02-03,117,4,False,2023,Final,Final,129,7,17,14391932,0.0,0.0,7.0,0.375,8.0,3.0,0.333333,9.0,3.0,0.000000,2.0,0.0,"{'id': 1038294, 'date': '2024-02-03', 'home_te...",33,0.0,1.0,"{'id': 38, 'first_name': 'Malik', 'height_feet...",9.0,7.0,2.0,"{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil...",1.0,38,17,Malik,Beasley,G,MIL
234535,1038294,2024-02-03,117,4,False,2023,Final,Final,129,7,17,14391933,8.0,1.0,3.0,1.000,5.0,5.0,0.909091,11.0,10.0,0.833333,6.0,5.0,"{'id': 1038294, 'date': '2024-02-03', 'home_te...",38,0.0,3.0,"{'id': 278, 'first_name': 'Damian', 'height_fe...",30.0,3.0,0.0,"{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil...",6.0,278,17,Damian,Lillard,G,MIL
234536,1038294,2024-02-03,117,4,False,2023,Final,Final,129,7,17,14391937,2.0,0.0,2.0,0.000,1.0,0.0,0.666667,6.0,4.0,0.000000,0.0,0.0,"{'id': 1038294, 'date': '2024-02-03', 'home_te...",21,0.0,3.0,"{'id': 112, 'first_name': 'Jae', 'height_feet'...",8.0,2.0,2.0,"{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil...",0.0,112,17,Jae,Crowder,F,MIL


### Re-arranging columns in the Dataframe

In [148]:
best_column_order = ['game_id', 'game_date', "season", 'new_home_team_id', 'new_vis_team_id',
                    'score_home','score_vis', 'team_id', 'abbreviation',
                     'player_id', 'position', 'first_name', 'last_name',
                      'pts', 'reb', 'stl',
         'ast', 'blk','oreb', 'dreb', 'fg3_pct', 'fg3a',
       'fg3m', 'fg_pct', 'fgm','fga',  'ft_pct', 'ftm', 'fta',
                     'turnover', 'pf', 'min','home_team_id', 'visitor_team_id',
                     'period', 'status', 'id', 'game', 'player', 'team']

merged_df = merged_df[best_column_order]

In [149]:
merged_df.drop(columns=["status", "period"])

,game_id,game_date,season,new_home_team_id,new_vis_team_id,score_home,score_vis,team_id,abbreviation,player_id,position,first_name,last_name,pts,reb,stl,ast,blk,oreb,dreb,fg3_pct,fg3a,fg3m,fg_pct,fgm,fga,ft_pct,ftm,fta,turnover,pf,min,home_team_id,visitor_team_id,id,game,player,team
0,47179,2019-01-30,2018,2,4,126,94,4,CHA,33,F-G,Nicolas,Batum,13.0,2.0,2.0,4.0,0.0,2.0,0.0,0.250,4.0,1.0,0.545000,6.0,11.0,0.000000,0.0,0.0,3.0,3.0,26:38,4,False,1076654,"{'id': 47179, 'date': '2019-01-30', 'home_team...","{'id': 33, 'first_name': 'Nicolas', 'height_fe...","{'id': 4, 'abbreviation': 'CHA', 'city': 'Char..."
1,47179,2019-01-30,2018,2,4,126,94,4,CHA,482,F,Marvin,Williams,10.0,2.0,1.0,0.0,0.0,0.0,2.0,1.000,2.0,2.0,1.000000,4.0,4.0,0.000000,0.0,0.0,0.0,0.0,27:23,4,False,1076655,"{'id': 47179, 'date': '2019-01-30', 'home_team...","{'id': 482, 'first_name': 'Marvin', 'height_fe...","{'id': 4, 'abbreviation': 'CHA', 'city': 'Char..."
2,47179,2019-01-30,2018,2,4,126,94,4,CHA,48,C,Bismack,Biyombo,2.0,5.0,0.0,2.0,1.0,1.0,4.0,0.000,0.0,0.0,0.333000,1.0,3.0,0.000000,0.0,0.0,1.0,1.0,13:34,4,False,1076656,"{'id': 47179, 'date': '2019-01-30', 'home_team...","{'id': 48, 'first_name': 'Bismack', 'height_fe...","{'id': 4, 'abbreviation': 'CHA', 'city': 'Char..."
3,47179,2019-01-30,2018,2,4,126,94,4,CHA,267,G,Jeremy,Lamb,6.0,5.0,0.0,1.0,0.0,0.0,5.0,0.400,5.0,2.0,0.286000,2.0,7.0,0.000000,0.0,0.0,2.0,2.0,23:10,4,False,1076657,"{'id': 47179, 'date': '2019-01-30', 'home_team...","{'id': 267, 'first_name': 'Jeremy', 'height_fe...","{'id': 4, 'abbreviation': 'CHA', 'city': 'Char..."
4,47179,2019-01-30,2018,2,4,126,94,4,CHA,465,G,Kemba,Walker,21.0,1.0,1.0,2.0,1.0,0.0,1.0,0.286,7.0,2.0,0.450000,9.0,20.0,1.000000,1.0,1.0,1.0,0.0,26:31,4,False,1076658,"{'id': 47179, 'date': '2019-01-30', 'home_team...","{'id': 465, 'first_name': 'Kemba', 'height_fee...","{'id': 4, 'abbreviation': 'CHA', 'city': 'Char..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234533,1038294,2024-02-03,2023,7,17,117,129,17,MIL,284,C,Robin,Lopez,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,1.000000,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,04,4,False,14391931,"{'id': 1038294, 'date': '2024-02-03', 'home_te...","{'id': 284, 'first_name': 'Robin', 'height_fee...","{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil..."
234534,1038294,2024-02-03,2023,7,17,117,129,17,MIL,38,G,Malik,Beasley,9.0,7.0,2.0,0.0,0.0,0.0,7.0,0.375,8.0,3.0,0.333333,3.0,9.0,0.000000,0.0,2.0,1.0,1.0,33,4,False,14391932,"{'id': 1038294, 'date': '2024-02-03', 'home_te...","{'id': 38, 'first_name': 'Malik', 'height_feet...","{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil..."
234535,1038294,2024-02-03,2023,7,17,117,129,17,MIL,278,G,Damian,Lillard,30.0,3.0,0.0,8.0,1.0,0.0,3.0,1.000,5.0,5.0,0.909091,10.0,11.0,0.833333,5.0,6.0,6.0,3.0,38,4,False,14391933,"{'id': 1038294, 'date': '2024-02-03', 'home_te...","{'id': 278, 'first_name': 'Damian', 'height_fe...","{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil..."
234536,1038294,2024-02-03,2023,7,17,117,129,17,MIL,112,F,Jae,Crowder,8.0,2.0,2.0,2.0,0.0,0.0,2.0,0.000,1.0,0.0,0.666667,4.0,6.0,0.000000,0.0,0.0,0.0,3.0,21,4,False,14391937,"{'id': 1038294, 'date': '2024-02-03', 'home_te...","{'id': 112, 'first_name': 'Jae', 'height_feet'...","{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil..."


### Creating Fantasy score

In [150]:
merged_df["fantasy_bonus"] = merged_df.apply(lambda row: row.loc['pts'] + row.loc['reb'] +row.loc['ast']+row.loc['stl']+ row.loc['fgm'] + row.loc['fg3m'] + row.loc['ftm'], axis=1)
merged_df["fantasy_malus"] = merged_df.apply(lambda row: row.loc['turnover'] + (row.loc["fga"] - row.loc["fgm"]) + (row.loc["fg3a"] - row.loc["fg3m"]) + (row.loc["fta"] - row.loc["ftm"]), axis=1)
merged_df["fantasy_score"] = merged_df["fantasy_bonus"] - merged_df["fantasy_malus"]
merged_df["display_last_name"] = merged_df["first_name"].str[0] + "." + merged_df["last_name"]

### Creating an impact table - In progress

In [154]:
pd.set_option('display.max_columns', None)

In [237]:
def create_impact_df(merged_df, given_date):
    # Convert the given date to datetime
    given_date = pd.to_datetime(given_date)

    # Define the start of the season
    season_start_date = pd.to_datetime('2023-10-24')

    # Filter the data for the season, last 30 days, and last 10 days
    season_df = merged_df[(merged_df['game_date'] >= season_start_date) & (merged_df['game_date'] <= given_date)]
    last_30_days_df = merged_df[(merged_df['game_date'] >= given_date - pd.Timedelta(days=30)) & (merged_df['game_date'] <= given_date)]
    last_10_days_df = merged_df[(merged_df['game_date'] >= given_date - pd.Timedelta(days=10)) & (merged_df['game_date'] <= given_date)]

    # Drop duplicates to avoid multiple counting during aggregation
    season_df = season_df.drop_duplicates(subset=['player_id', 'game_id'])
    last_30_days_df = last_30_days_df.drop_duplicates(subset=['player_id', 'game_id'])
    last_10_days_df = last_10_days_df.drop_duplicates(subset=['player_id', 'game_id'])

    # Group by player_id and sum the statistics for each period
    season_agg = season_df.groupby('player_id').agg({
        'display_last_name': 'first',
        'ast': 'sum'

    }).rename(columns={'ast': 'ast_seas',
                       'display_last_name': 'player_name'})

    last_30_agg = last_30_days_df.groupby('player_id').agg({
        'ast': 'sum'
    }).rename(columns={'ast': 'ast_30'})

    last_10_agg = last_10_days_df.groupby('player_id').agg({
        'ast': 'sum'
    }).rename(columns={'ast': 'ast_10'})

    # Merge the aggregated data
    impact_table = season_agg.merge(last_30_agg, on='player_id', how='left').merge(last_10_agg, on='player_id', how='left')

    # Fill NaN values with 0 (in case some players don't have data for the last 30 or 10 days)
    impact_table = impact_table.fillna(0)

    # Sort by descending ast_seas
    impact_table = impact_table.sort_values('ast_seas', ascending=False).reset_index(drop=True)

    # Round values to 1 decimal place
    impact_table = impact_table.round({'ast_seas': 1, 'ast_30': 1, 'ast_10': 1})

    return impact_table


In [238]:
# Example usage:
given_date = '2023-11-30'
impact_table = create_impact_df(merged_df, given_date)

In [239]:
impact_table.head(50)

,player_name,ast_seas,ast_30,ast_10
0,T.Young,121.0,81.0,50.0
1,F.VanVleet,82.0,61.0,27.0
2,C.Paul,81.0,74.0,22.0
3,T.Haliburton,78.0,65.0,42.0
4,L.James,76.0,68.0,33.0
5,R.Westbrook,76.0,60.0,39.0
6,J.Brunson,75.0,65.0,32.0
7,N.Jokic,75.0,63.0,35.0
8,D.Lillard,74.0,65.0,41.0
9,C.Cunningham,74.0,60.0,5.0


# Prochaines étapes :

In [29]:
#duplicates have been removed
#matches manquants dans données initiales - est-ce dans un merge ou dans les données initiales?
#faire différents fantasy scores pour pour 30, 15 et 7 jours
